## 1.Download Dataset

In [1]:
#import libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

In [2]:
#import the canada_coordinate.csv table as previously created
canada_df = pd.read_csv('canada_coordinate.csv')
canada_df.head(10) #display the first 10 rows from the table

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696665,-79.260163


## 2.Creating Toronto Dataframe

#### For this exercise, only neighborhoods in Toronto is explored. So, the original dataframe is sliced to create new dataframe of Toronto.

In [3]:
# explore the all the Borough list
canada_df['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [4]:
# creating a list of Toronto from the Borough
toronto = ['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [5]:
#creating Toronto dataframe from the original dataframe by using the Toronto list created
toronto_df = canada_df[canada_df.Borough.isin(toronto)].reset_index(drop = True)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676845,-79.295225
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314673
3,M4M,East Toronto,Studio District,43.662766,-79.334830
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085
5,M4P,Central Toronto,Davisville North,43.712815,-79.388526
6,M4R,Central Toronto,North Toronto West,43.714523,-79.406960
7,M4S,Central Toronto,Davisville,43.703395,-79.385964
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.690655,-79.383561
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686083,-79.402335


#### Checking the dataframe size

In [6]:
toronto_df.shape 

(38, 5)

#### Getting Geographical Coordinate for Toronto

In [7]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent = 'ca_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('Geography coordinate of Toronto is {}, {}'.format(latitude, longitude))

Geography coordinate of Toronto is 43.653963, -79.387207


#### Visualizing Neighborhoods in Toronto

In [8]:
# create toronto map
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 11)

# add markers to the toronto map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, lng],
                        radius = 5,
                        popup = label,
                        color = 'blue',
                        fill = True,
                        fill_color = '#3186cc',
                        fill_opacity = 0.7,
                        parse_html = False,
                       ).add_to(map_toronto)
map_toronto

## 3.Explore Neighborhoods in Toronto

#### Define Foursquare Credentials and Version

In [9]:
CLIENT_ID = 'XQYTD01WFJPCVBX01GCGY5GJWJ5DH05PIPAZEHJFVESOVNR0'
CLIENT_SECRET = 'BI2NPYJKK0FOBFTDDFT0DPFW2YDKEOFEP10J4R1VE5SF4PL5'
VERSION = '20180605'

print('Client_ID: {}, Client_secret: {}, Version: {}'.format(CLIENT_ID, CLIENT_SECRET, VERSION))

Client_ID: XQYTD01WFJPCVBX01GCGY5GJWJ5DH05PIPAZEHJFVESOVNR0, Client_secret: BI2NPYJKK0FOBFTDDFT0DPFW2YDKEOFEP10J4R1VE5SF4PL5, Version: 20180605


#### create a function to get nearby venue

In [10]:
LIMIT = 100 # limiting the results for 100 for each neighborhood

# in this exercise, the radius used is 1000 
def getNearbyVenues(names, latitudes, longitudes, radius = 1000): 
    
    venues_list =[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        #create API request for URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT
        )
        
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevent information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venues_list in venues_list for item in venues_list])
    nearby_venues.columns = ['Neighborhood',
                                 'Neighborhood Latitude',
                                 'Neighborhood Longitude',
                                 'Venue',
                                 'Venue Latitude',
                                 'Venue Longitude',
                                 'Venue Category'
                                ]
        
    return (nearby_venues)

#### Create new dataframe *toronto_venue* from the above function

In [11]:
toronto_venues = getNearbyVenues(names = toronto_df['Neighborhood'], 
                                 latitudes = toronto_df['Latitude'], 
                                 longitudes = toronto_df['Longitude'])

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

#### Checking the dataframe size and the new dataframe created

In [12]:
print(toronto_venues.shape)
toronto_venues.head(10)

(3172, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676845,-79.295225,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676845,-79.295225,Glen Manor Ravine,43.676821,-79.293942,Trail
2,The Beaches,43.676845,-79.295225,The Beech Tree,43.680493,-79.288846,Gastropub
3,The Beaches,43.676845,-79.295225,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
4,The Beaches,43.676845,-79.295225,Beaches Bake Shop,43.680363,-79.289692,Bakery
5,The Beaches,43.676845,-79.295225,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
6,The Beaches,43.676845,-79.295225,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
7,The Beaches,43.676845,-79.295225,Mastermind Toys,43.671453,-79.293971,Toy / Game Store
8,The Beaches,43.676845,-79.295225,Bagels On Fire,43.672864,-79.286784,Bagel Shop
9,The Beaches,43.676845,-79.295225,Kew Gardens,43.669038,-79.298538,Park


#### Checking numbers of venues returned for each Neighborhood

In [13]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton, Exhibition Place, Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",100,100,100,100,100,100
"Cabbagetown, St. James Town",39,39,39,39,39,39
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100


#### Check how many unique categories can be curated from all the returned venues

In [14]:
print('There are {} unique categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 287 unique categories


## 4.Analyze Each Neighborhood

In [15]:
#using onehot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot.drop('Neighborhood', axis = 1, inplace = True)
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] # adding neighborhood back to the dataframe

# move the neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(10)

,Neighborhood,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Organic Grocery,Other Great Outdoors,Other Nightlife,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Win

#### Checking the dataframe size

In [16]:
toronto_onehot.shape

(3172, 287)

#### Group the row by Neighborhood and calculating mean of the frequency occurance foreach category

In [17]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

,Neighborhood,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Organic Grocery,Other Great Outdoors,Other Nightlife,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Win

#### Checking the dataframe size

In [18]:
toronto_grouped.shape

(38, 287)

#### Print each Neighborhood with the 5 top most common venues

In [19]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("---"+hood+"---")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---Adelaide, King, Richmond---
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.08
2                Hotel  0.07
3  Japanese Restaurant  0.04
4              Theater  0.04


---Berczy Park---
         venue  freq
0  Coffee Shop  0.10
1         Café  0.06
2        Hotel  0.05
3   Restaurant  0.04
4     Beer Bar  0.04


---Brockton, Exhibition Place, Parkdale Village---
         venue  freq
0         Café  0.07
1  Coffee Shop  0.07
2          Bar  0.05
3   Restaurant  0.04
4       Bakery  0.04


---Business Reply Mail Processing Centre 969 Eastern---
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.06
2                Hotel  0.06
3  American Restaurant  0.03
4                  Bar  0.03


---CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara---
                venue  freq
0         Coffee Shop  0.11
1  Italian Restaurant  0.05
2          Restaurant  0.05
3         Y

#### Create funtion to sort the venues in ascending order

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create dataframe that show the 10 most common venues of each neighborhood

In [21]:
# set number of venues to create
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to the number of top venues
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create new dataframe 
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Theater,Japanese Restaurant,American Restaurant,Sushi Restaurant,Concert Hall,Bakery,Cosmetics Shop
1,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Beer Bar,Park,Cocktail Bar,Bakery,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Bar,Restaurant,Bakery,Furniture / Home Store,Gift Shop,Beer Bar,Ice Cream Shop,Sandwich Place
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Café,Hotel,Pizza Place,Bar,Italian Restaurant,American Restaurant,Concert Hall,Deli / Bodega,Asian Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Restaurant,Italian Restaurant,Yoga Studio,Bar,Spa,Café,Park,Gym,French Restaurant
5,"Cabbagetown, St. James Town",Park,Gastropub,Diner,Café,Japanese Restaurant,Pool,Filipino Restaurant,Italian Restaurant,Pet Store,Jewelry Store
6,Central Bay Street,Coffee Shop,Café,Clothing Store,Vegetarian / Vegan Restaurant,Italian Restaurant,Japanese Restaurant,Ramen Restaurant,Tea Room,Cosmetics Shop,Park
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Bakery,Yoga Studio,Bar,Dessert Shop,Mexican Restaurant,Coffee Shop,Furniture / Home Store,Ice Cream Shop
8,Christie,Korean Restaurant,Café,Grocery Store,Coffee Shop,Park,Indian Restaurant,Japanese Restaurant,Cocktail Bar,Pizza Place,Sushi Restaurant
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Pizza Place,Dance Studio,Gym,Mediterranean Restaurant,Men's Store,Gastropub


## 5.Clustering The Neighborhood

#### Using K-Means to cluster Neighborhood into 5 clusters

In [22]:
#set number od clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis = 1)

# run the kmean clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 2).fit(toronto_grouped_clustering)

# check the clutsering cloumn created for top 10 row
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 0, 2, 0, 0, 2], dtype=int32)

#### Create dataframe which include clustering label and top 10 most common venues for each neighborhood

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_df with the neighborhpood_venues_sorted to get the latitude and longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')

toronto_merged.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676845,-79.295225,0,Pub,Coffee Shop,Park,Breakfast Spot,Bar,Pizza Place,Thai Restaurant,Caribbean Restaurant,Burger Joint,Salon / Barbershop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120,2,Greek Restaurant,Coffee Shop,Café,Pub,Bakery,Italian Restaurant,Fast Food Restaurant,Pizza Place,Sandwich Place,Grocery Store
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314673,0,Indian Restaurant,Coffee Shop,Beach,Pub,Pizza Place,Fast Food Restaurant,Sandwich Place,Burrito Place,Café,Restaurant
3,M4M,East Toronto,Studio District,43.662766,-79.334830,0,Coffee Shop,Pizza Place,Café,Bar,Bakery,Italian Restaurant,American Restaurant,Park,Grocery Store,Sushi Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085,3,Bookstore,Gym / Fitness Center,Park,Bus Line,Trail,Café,Pharmacy,Restaurant,Coffee Shop,Electronics Store
5,M4P,Central Toronto,Davisville North,43.712815,-79.388526,0,Coffee Shop,Pizza Place,Italian Restaurant,Café,Fast Food Restaurant,Dessert Shop,Sushi Restaurant,Pharmacy,Gym,Supermarket
6,M4R,Central Toronto,North Toronto West,43.714523,-79.406960,0,Sporting Goods Shop,Italian Restaurant,Coffee Shop,Skating Rink,Café,Mexican Restaurant,Diner,Park,Burger Joint,Salon / Barbershop
7,M4S,Central Toronto,Davisville,43.703395,-79.385964,0,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Coffee Shop,Café,Restaurant,Pizza Place,Fast Food Restaurant,Bakery,Sandwich Place
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.690655,-79.383561,0,Coffee Shop,Italian Restaurant,Grocery Store,Park,Gym,Thai Restaurant,Bank,Sushi Restaurant,Bagel Shop,Sandwich Place
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686083,-79.402335,0,Coffee Shop,Sushi Restaurant,Italian Restaurant,Café,Sandwich Place,Thai Restaurant,Park,Gym,Pizza Place,Burger Joint


In [24]:
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676845,-79.295225,0,Pub,Coffee Shop,Park,Breakfast Spot,Bar,Pizza Place,Thai Restaurant,Caribbean Restaurant,Burger Joint,Salon / Barbershop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120,2,Greek Restaurant,Coffee Shop,Café,Pub,Bakery,Italian Restaurant,Fast Food Restaurant,Pizza Place,Sandwich Place,Grocery Store
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314673,0,Indian Restaurant,Coffee Shop,Beach,Pub,Pizza Place,Fast Food Restaurant,Sandwich Place,Burrito Place,Café,Restaurant
3,M4M,East Toronto,Studio District,43.662766,-79.334830,0,Coffee Shop,Pizza Place,Café,Bar,Bakery,Italian Restaurant,American Restaurant,Park,Grocery Store,Sushi Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085,3,Bookstore,Gym / Fitness Center,Park,Bus Line,Trail,Café,Pharmacy,Restaurant,Coffee Shop,Electronics Store
5,M4P,Central Toronto,Davisville North,43.712815,-79.388526,0,Coffee Shop,Pizza Place,Italian Restaurant,Café,Fast Food Restaurant,Dessert Shop,Sushi Restaurant,Pharmacy,Gym,Supermarket
6,M4R,Central Toronto,North Toronto West,43.714523,-79.406960,0,Sporting Goods Shop,Italian Restaurant,Coffee Shop,Skating Rink,Café,Mexican Restaurant,Diner,Park,Burger Joint,Salon / Barbershop
7,M4S,Central Toronto,Davisville,43.703395,-79.385964,0,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Coffee Shop,Café,Restaurant,Pizza Place,Fast Food Restaurant,Bakery,Sandwich Place
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.690655,-79.383561,0,Coffee Shop,Italian Restaurant,Grocery Store,Park,Gym,Thai Restaurant,Bank,Sushi Restaurant,Bagel Shop,Sandwich Place
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686083,-79.402335,0,Coffee Shop,Sushi Restaurant,Italian Restaurant,Café,Sandwich Place,Thai Restaurant,Park,Gym,Pizza Place,Burger Joint


#### Visualizing the map of the resulting clusters

In [25]:
# create map
map_clusters = folium.Map(location = [latitude,longitude], zoom_start=11)

#setting the color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html= True)
    folium.CircleMarker ([lat, lon],
                         radius = 5,
                         popup = label,
                         color = rainbow[cluster - 1],
                         fill = True,
                         fill_color = rainbow[cluster - 1],
                         fill_opacity = 0.7
                        ).add_to(map_clusters)
    
map_clusters

## 6.Examining Each Cluster

#### Examining each cluster to determine the venues categories that distinguish each cluster.

### Cluster 1

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Pub,Coffee Shop,Park,Breakfast Spot,Bar,Pizza Place,Thai Restaurant,Caribbean Restaurant,Burger Joint,Salon / Barbershop
2,East Toronto,0,Indian Restaurant,Coffee Shop,Beach,Pub,Pizza Place,Fast Food Restaurant,Sandwich Place,Burrito Place,Café,Restaurant
3,East Toronto,0,Coffee Shop,Pizza Place,Café,Bar,Bakery,Italian Restaurant,American Restaurant,Park,Grocery Store,Sushi Restaurant
5,Central Toronto,0,Coffee Shop,Pizza Place,Italian Restaurant,Café,Fast Food Restaurant,Dessert Shop,Sushi Restaurant,Pharmacy,Gym,Supermarket
6,Central Toronto,0,Sporting Goods Shop,Italian Restaurant,Coffee Shop,Skating Rink,Café,Mexican Restaurant,Diner,Park,Burger Joint,Salon / Barbershop
7,Central Toronto,0,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Coffee Shop,Café,Restaurant,Pizza Place,Fast Food Restaurant,Bakery,Sandwich Place
8,Central Toronto,0,Coffee Shop,Italian Restaurant,Grocery Store,Park,Gym,Thai Restaurant,Bank,Sushi Restaurant,Bagel Shop,Sandwich Place
9,Central Toronto,0,Coffee Shop,Sushi Restaurant,Italian Restaurant,Café,Sandwich Place,Thai Restaurant,Park,Gym,Pizza Place,Burger Joint
11,Downtown Toronto,0,Park,Gastropub,Diner,Café,Japanese Restaurant,Pool,Filipino Restaurant,Italian Restaurant,Pet Store,Jewelry Store
19,Downtown Toronto,0,Harbor / Marina,Bar,Airport Lounge,Airport Service,Airport Terminal,Burger Joint,Coffee Shop,Nudist Beach,Sculpture Garden,Music Venue


### Cluster 2

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Pharmacy,Café,Skating Rink,Trail,Bank,Dry Cleaner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


### Cluster 3

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,2,Greek Restaurant,Coffee Shop,Café,Pub,Bakery,Italian Restaurant,Fast Food Restaurant,Pizza Place,Sandwich Place,Grocery Store
12,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Pizza Place,Dance Studio,Gym,Mediterranean Restaurant,Men's Store,Gastropub
13,Downtown Toronto,2,Coffee Shop,Café,Park,Restaurant,Theater,Italian Restaurant,Breakfast Spot,Bakery,Gym / Fitness Center,Thai Restaurant
14,Downtown Toronto,2,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Gastropub,Tea Room,Café,Italian Restaurant,Diner,Park,Bookstore
15,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Hotel,Gastropub,Bakery,Cosmetics Shop,Italian Restaurant,Breakfast Spot,Seafood Restaurant
16,Downtown Toronto,2,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Beer Bar,Park,Cocktail Bar,Bakery,Farmers Market
17,Downtown Toronto,2,Coffee Shop,Café,Clothing Store,Vegetarian / Vegan Restaurant,Italian Restaurant,Japanese Restaurant,Ramen Restaurant,Tea Room,Cosmetics Shop,Park
18,Downtown Toronto,2,Coffee Shop,Café,Hotel,Theater,Japanese Restaurant,American Restaurant,Sushi Restaurant,Concert Hall,Bakery,Cosmetics Shop
20,Downtown Toronto,2,Coffee Shop,Hotel,Café,Lounge,Restaurant,Concert Hall,Steakhouse,Italian Restaurant,American Restaurant,Bakery
21,Downtown Toronto,2,Coffee Shop,Café,Hotel,Restaurant,Gastropub,American Restaurant,Beer Bar,Japanese Restaurant,Steakhouse,Bakery


### Cluster 4

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Bookstore,Gym / Fitness Center,Park,Bus Line,Trail,Café,Pharmacy,Restaurant,Coffee Shop,Electronics Store


### Cluster 5

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,4,Park,Trail,Italian Restaurant,Building,Athletics & Sports,Beer Store,Gourmet Shop,Bank,Sporting Goods Shop,Gym / Fitness Center
